# FEMA Disaster Data Analysis

This notebook explores the FEMA Disaster Declarations dataset to understand patterns and inform our prediction model.

First, let's set up our environment and fetch the data.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn requests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
import json

# Set plotting style
plt.style.use('seaborn')
sns.set_palette("husl")

In [ ]:
# Fetch data directly from FEMA API
def fetch_fema_data():
    print("Fetching data from FEMA API...")
    url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["DisasterDeclarationsSummaries"])
        return df
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Load the data
try:
    df = fetch_fema_data()
    print("Data successfully loaded!")
    
    # Display basic information
    print("\nDataset Shape:", df.shape)
    print("\nColumns:", df.columns.tolist())
    print("\nData Types:\n", df.dtypes)
    print("\nMissing Values:\n", df.isnull().sum())
except Exception as e:
    print(f"Error loading data: {str(e)}")

## 2. Temporal Analysis

In [ ]:
# Convert date columns
date_columns = ['declarationDate', 'incidentBeginDate', 'incidentEndDate']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Analyze disasters over time
plt.figure(figsize=(15, 6))
df.groupby(df['declarationDate'].dt.year)['incidentType'].count().plot(kind='line')
plt.title('Number of Disaster Declarations by Year')
plt.xlabel('Year')
plt.ylabel('Number of Declarations')
plt.show()

# Seasonal patterns
plt.figure(figsize=(15, 6))
df.groupby(df['declarationDate'].dt.month)['incidentType'].count().plot(kind='bar')
plt.title('Disaster Declarations by Month')
plt.xlabel('Month')
plt.ylabel('Number of Declarations')
plt.show()

## 3. Geographic Analysis

In [ ]:
# Rest of the notebook content remains the same...
# (Geographic Analysis, Disaster Type Analysis, etc.)